In [1]:
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.exc import GeocoderTimedOut
import pandas as pd


# definire il path in cui è stato clonato il repository
path =r"/mnt/c/Users/denise/Sviluppo/Repos/Address_verifier/"
import sys
sys.path.append(path)
import functions as mav


Defined function that take two lists of addresses and return distance and geographic coordinates (like latitude and longitude)


In [ ]:
dati = {"indirizzo_1" : ["Via Roma, Milano", "Corso Marconi, Genova", "Piazza Della Vittoria, Genova"],
        "indirizzo_2": ["Via Nicolò Paganini, Spezia", "Località Piazzano", "Località Gracchia"]}
df_ind = pd.DataFrame(dati)
df_ind['ID'] = range(1, len(df_ind)+1)
df_ind

In [15]:
lista_df_1 = df_ind["indirizzo_1"].tolist()
lista_df_2 = df_ind["indirizzo_2"].tolist()

In [8]:
l1 = ['Piazza San Marco, Venezia', 'Oxford Street, London']
l2 = ['Via Marsala, Roma', 'Via Nino Bixio, Genova']


In [ ]:
try:
    distances, coordinates = mav.distance_coordinate_addresses(lista_df_1,lista_df_2)
    for i, (distance, (c1, c2)) in enumerate(zip(distances, coordinates), start=1):
        print(f"Distanza tra gli indirizzi {i}: {distance:.2f} km")
        print(f"Coordinate Indirizzo {i} - Lista 1: {c1}")
        print(f"Coordinate Indirizzo {i} - Lista 2: {c2}")
except ValueError as e:
    print(e)


Defined the map center based on geographic coordinates\
The map returns the distance in beeline between pairs of addresses. 

In [ ]:
mav.mappa_distanze_geo(coordinates)

In [ ]:
import os
print(f"File salvato in: {os.getcwd()}")